# EXAMPLES (RAG)
- [RAG](https://docs.activeloop.ai/examples/rag)
  - [RAG Quickstart](https://docs.activeloop.ai/examples/rag/quickstart)
  - [RAG Tutorials](https://docs.activeloop.ai/examples/rag/tutorials)
    - [Vector Store Basics](https://docs.activeloop.ai/examples/rag/tutorials/vector-store-basics)
    - [Vector Search Options](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options)
      - [LangChain API](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options/langchain-api)
      - [Deep Lake Vector Store API](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options/vector-store-api)
      - [Managed Database REST API](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options/rest-api)
    - [Customizing Your Vector Store](https://docs.activeloop.ai/examples/rag/tutorials/step-4-customizing-vector-stores)
    - [Image Similarity Search](https://docs.activeloop.ai/examples/rag/tutorials/image-similarity-search)
    - [Improving Search Accuracy using Deep Memory](https://docs.activeloop.ai/examples/rag/tutorials/deepmemory)
  - [LangChain Integration](https://docs.activeloop.ai/examples/rag/langchain-integration)
  - [**LlamaIndex Integration**](https://docs.activeloop.ai/examples/rag/llamaindex-integration)
  - [Managed Tensor Database](https://docs.activeloop.ai/examples/rag/managed-database)
    - [REST API](https://docs.activeloop.ai/examples/rag/managed-database/rest-api)
    - [Migrating Datasets to the Tensor Database](https://docs.activeloop.ai/examples/rag/managed-database/migrating-datasets-to-the-tensor-database)
  - [Deep Memory](https://docs.activeloop.ai/examples/rag/deep-memory)
    - [How it Works](https://docs.activeloop.ai/examples/rag/deep-memory/how-it-works)

## RAG (LlamaIndex Integration)

### Use Deep Lake as a Vector Store in LlamaIndex
*Deep Lake can be used as a VectorStore in LlamaIndex for building Apps that require filtering and vector search. In this tutorial we will show how to create a Deep Lake Vector Store in LangChain and use it to build a Q&A App about the Twitter OSS recommendation algorithm.*

In [ ]:
# %pip3 install llama-index-vector-stores-deeplake
# !pip3 install langchain llama-index deeplake

In [5]:
# %pip install llama-index-vector-stores-deeplake
# !pip install llama-index

#### Downloading and Preprocessing the Data

In [1]:
import os
import textwrap

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core import StorageContext

C:\Users\Pavel\projects\ai-llm-deeplake\.venv\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
from dotenv import load_dotenv

load_dotenv(override = True)
open_api_key = os.getenv('OPENAI_API_KEY')
activeloop_token = os.getenv('ACTIVELOOP_TOKEN')

MODEL_GPT = 'gpt-4o-mini'

In [3]:
# Clone the Twitter OSS recommendation algorithm

# !git clone https://github.com/twitter/the-algorithm

In [4]:
# Specify a local path to the files and add a reader for processing and chunking them

# repo_path = '/the-algorithm'
# repo_path = './the-algorithm'
repo_path = './the-algorithm/twml/twml/layers'

documents = SimpleDirectoryReader(repo_path, recursive=True).load_data()

In [5]:
print(type(documents))
print(len(documents))

<class 'list'>
14


#### Creating the Deep Lake Vector Store

In [6]:
# Create an empty Deep Lake Vector Store using a specified path

# dataset_path = 'hub://<org-id>/twitter_algorithm'
dataset_path = 'hub://pavelkloscz/twitter_algorithm_twml_2'  # [twml] subdirectory of this github repo
vector_store = DeepLakeVectorStore(dataset_path=dataset_path)

Your Deep Lake dataset has been successfully created!


In [7]:
# Deep Lake Vector Store has 4 tensors including the text, embedding, ids, and  metadata which includes the filename of the text

 #  tensor      htype     shape    dtype  compression
 #  -------    -------   -------  -------  ------- 
 #   text       text      (0,)      str     None   
 # metadata     json      (0,)      str     None   
 # embedding  embedding   (0,)    float32   None   
 #    id        text      (0,)      str     None  

In [8]:
# Create a LlamaIndex StorageContext and VectorStoreIndex, and use the from_documents() method to populate
#   the Vector Store with data. This step takes several minutes because of the time to embed the text

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,
)

# [OUTPUT] Vector Store has 8286 rows of data
 #  tensor      htype       shape       dtype  compression
 #  -------    -------     -------     -------  ------- 
 #   text       text      (8262, 1)      str     None   
 # metadata     json      (8262, 1)      str     None   
 # embedding  embedding  (8262, 1536)  float32   None   
 #    id        text      (8262, 1)      str     None 

Uploading data to deeplake dataset.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 21.35it/s]
\

Dataset(path='hub://pavelkloscz/twitter_algorithm_twml_2', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (24, 1)      str     None   
 metadata     json      (24, 1)      str     None   
 embedding  embedding  (24, 1536)  float32   None   
    id        text      (24, 1)      str     None   


#### Use the Vector Store in a Q&A App

In [9]:
# Use the VectorStore in Q&A app, where the embeddings will be used to filter relevant documents (texts)
#   that are fed into an LLM in order to answer a question.
# If we were on another machine, we would load the existing Vector Store without re-ingesting the data

vector_store = DeepLakeVectorStore(dataset_path=dataset_path, read_only=True)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

Deep Lake Dataset in hub://pavelkloscz/twitter_algorithm_twml_2 already exists, loading from the storage


In [10]:
# Create the LlamaIndex query engine and run a query

query_engine = index.as_query_engine()

In [11]:
# response = query_engine.query("What programming language is most of the SimClusters written in?")
response = query_engine.query("What programming language is most of the Batch Prediction written in?")
print(str(response))

Python


In [12]:
response = query_engine.query("What programming language is most of the TWML written in?")
print(str(response))

Most of the TWML (TensorFlow Wrapper for Machine Learning) code is written in Python.
